# Question 3: Trajectory Evaluation and g2o

_Refer to the example notebooks for installation instructions_

# Evo

So you've implemented 2D SLAM, great! Now, what? We need a measure of how good the trajectory is. The error/loss used earlier doesn't tell us much about how the trajectory differs from the ground truth. Here, we try to do just this - compute error metrics. Rather than computing these from scratch, we will just Evo - https://github.com/MichaelGrupp/evo/.

Look at the absolute pose error (APE) and relative pose error (RPE). What do they capture and how are they calculated (descriptive answer)? How do these metrics differ in methodology? Can we determine if the error is more along the x/y axis?

Answer the above questions and report errors for the obtained trajectory.

In [ ]:
# Write up with plots/images

If you're interested, play around with this tool and add any other plots that you think might be relevant/interesting.

## Analysis using Evo

Analysis of the trajectory generated in question 2 is explored in this section. The method, commands, inference, etc. are also explored


### Setup and Generation

To install [Evo](https://github.com/MichaelGrupp/evo/) the following command was run

```bash
pip install evo
```

After this, binary files get generated in the `bin` folder of the Python environment. Post this, the following commands were run

#### Generating Kitti file for Ground Truth

The following commands were run to generate the ground truth [.kitti](https://github.com/MichaelGrupp/evo/wiki/Formats#kitti---kitti-dataset-pose-format) file. The starting path is the project root (folder for `Project1`)

```bash
cd misc
cp ../data/gt.txt ./ground-truth.g2o
python ./g2o_to_kitti.py ground-truth.g2o gt.kitti
```

This generates a `gt.kitti` that contains the ground truth poses as transforms of the form `[R|t]` (12 numbers per row as described in format).


#### Generating Kitti file for calibrated data

The calibrated data generated as a result of Question 2 is converted to a `.kitti` file using the following commands (in the same `misc` folder as before)

```bash
cp ../questions/results/q2/edges-calibrated.g2o ./
python ./g2o_to_kitti.py edges-calibrated.g2o opt.kitti
```

This generates a `opt.kitti` file that contains calibrated poses as shown in the image below. This is compared in the next section.

![Calibrated Trajectory](./results/q2/p2/lm-out-1.png)

### Trajectory analysis

This is done using the following command

```bash
evo_traj kitti gt.kitti opt.kitti -v --plot --plot_mode xy
```



The resultant output is

```txt
--------------------------------------------------------------------------------
Loaded 120 poses from: gt.kitti
Loaded 120 poses from: opt.kitti
--------------------------------------------------------------------------------
name:	gt
infos:
	nr. of poses	120
	path length (m)	52.976517997999295
	pos_end (m)	[-2.8 -4.5  0. ]
	pos_start (m)	[-8.  5.  0.]
--------------------------------------------------------------------------------
name:	opt
infos:
	nr. of poses	120
	path length (m)	54.66919770066222
	pos_end (m)	[-0.63074972  1.07208598  0.        ]
	pos_start (m)	[-8.00001143  4.99998875  0.        ]
```

The errors seem big (in meters), at least the path length is more or less the same. The figures generated are shown here and explained


#### Trajectory Plot

![Trajectory plot](./results/q3/evo/traj_1.png)

This shows the ground truth and the optimal trajectory (obtained through `question 2`) in comparison,


#### XYZ Plot

The plot is shown below

![XYZ plot](./results/q3/evo/traj_2.png)

It shows the X, Y and Z values of the points in the ground truth and optimal trajectory. As expected, the optimal values aren't exactly matching, but they're closely following the ground truth (sometimes not very closely). This is the result of optimization.


#### RPY View

The plot is shown below

![RPY plot](./results/q3/evo/traj_3.png)

As expected, there is only yaw used here. Even here, the optimized trajectory is more or less following (roughly) the ground truth. However, as described in question 2, there appears to be errors in this (the ground truth), which cannot be fit easily.


### Relative Pose Error

This is done using the following command

```bash
evo_rpe kitti gt.kitti opt.kitti -v --plot --plot_mode xy
```


The resultant output is

```txt
--------------------------------------------------------------------------------
Loaded 120 poses from: gt.kitti
Loaded 120 poses from: opt.kitti
--------------------------------------------------------------------------------
Found 119 pairs with delta 1 (frames) among 120 poses using consecutive pairs.
Compared 119 relative pose pairs, delta = 1 (frames) with consecutive pairs.
Calculating RPE for translation part pose relation...
--------------------------------------------------------------------------------
RPE w.r.t. translation part (m)
for delta = 1 (frames) using consecutive pairs
(not aligned)

       max	0.292755
      mean	0.118914
    median	0.112302
       min	0.001513
      rmse	0.134819
       sse	2.162955
       std	0.063526

--------------------------------------------------------------------------------
```

The relative pose error is over a fixed time interval. It being less than a meter is slightly acceptable.

#### Plots

The plots are shown below

![RPE over trajectory](./results/q3/evo/rpe_2.png)
![RPE plot](./results/q3/evo/rpe_1.png)

The relative error is less (because it's piecewise in time, not accumulated). The graph also shows that it's not steady (it bounces all over the place). Note that this does not take alignment into consideration. However, there may be problems in the trajectory because of the fault in observations too (like the orientations not being proper).


### Absolute Pose Error

This is done using the following command

```bash
evo_ape kitti gt.kitti opt.kitti -v --plot --plot_mode xy
```


The output is shown below

```txt
--------------------------------------------------------------------------------
Loaded 120 poses from: gt.kitti
Loaded 120 poses from: opt.kitti
--------------------------------------------------------------------------------
Compared 120 absolute pose pairs.
Calculating APE for translation part pose relation...
--------------------------------------------------------------------------------
APE w.r.t. translation part (m)
(not aligned)

       max	6.801272
      mean	3.782564
    median	4.355961
       min	0.000016
      rmse	4.370817
       sse	2292.484714
       std	2.190033

--------------------------------------------------------------------------------
```

This figure is not very good. The track itself is around 15 meters. A 7 meter error is very bad on the surface.




The plots are shown below

![Absolute error plot](./results/q3/evo/ape_1.png)
![Absolute error over trajectory](./results/q3/evo/ape_2.png)

It can be seen that the absolute error increases with the trajectory for the first loop. Note that though it doesn't take alignment, due to the nature of the motion model, if the alignment goes off, it **does** creep into the absolute error more.

After the first loop is done, it decreases (as it comes back and loop closure starts). It then increases again as the trajectory continues.

# g2o

Install g2o as mentioned in `examples/g2o.ipynb` and optimise `edges.txt`, the file you used earlier. Also use `g2o_viewer` and optimize `intel` (a trajectory in the Intel research lab) and `sphere`. They should look something like:


<table><tr>
<td> <img src="../misc/intel.jpg" alt="Drawing" style="width: 250px;"/> </td>
<td> <img src="../misc/sphere.jpg" alt="Drawing" style="width: 250px;"/> </td>
</tr></table>

Write briefly about your observations and try out few options in the GUI. What do they do, how do they perform?

In [ ]:
# Your answer

## Installing g2o

Installing `g2o` executables is explored here


This is done by running the following commands

```bash
sudo apt update && sudo apt upgrade -y
```

Install requirements
```bash
sudo apt install cmake -y
sudo apt install libeigen3-dev -y
```

This next one is optional
```bash
sudo apt install libsuitesparse-dev qtdeclarative5-dev qt5-qmake libqglviewer-dev-qt5
```

Clone the repository
```bash
cd ~/repos/
git clone https://github.com/RainerKuemmerle/g2o.git
cd ./g2o/
```

Build everything
```bash
mkdir build && cd ./build/
cmake ../
make
```

The resultant executables are stored in the `bin` folder and the libraries are stored in the `lib` folder (in the `g2o` home folder). To add the executables to the `PATH`, the following was added to the `~/.bashrc`

```txt
repo_folder="${HOME}/repos"
g2o_folder="${repo_folder}/g2o"
g2o_bin_folder="${g2o_folder}/bin"
export PATH="${g2o_bin_folder}:$PATH"
```

The the following was run to source and verify

```bash
source ~/.zshrc
echo $PATH | sed -e 's/:/\n/g'
```

The installation is now complete

## Generating Optimized Trajectory file

The question 2 execution has yielded the `edges-poses.g2o` file that contains the vertices (poses) after applying the odometry / control sequentially. This has to be appended with the odometry constraints and loop closure constraints as described in `edges.txt`. This task is explained in this section. Note that we're not using the resulting / optimized vertices anywhere in this section (`g2o` will optimize).

Reference
- g2o file format on [GitHub Wiki](https://github.com/RainerKuemmerle/g2o/wiki/File-Format)

First, create a directory for everything done in this `g2o` section

```bash
cd ./questions/
mkdir g2o_exp && cd g2o_exp
```

Now, copy the two files to this folder

```bash
cp ../edges-poses.g2o ./
cp ../../data/edges.txt ./edges.g2o
```

Now, the files have to be merged

1. Delete the first line of `edges.g2o`
2. Concatenate `edges-poses.g2o` with the new contents of `edges.g2o`
3. Save the resulting file as `given.g2o`

This operations are easy using Vim editor, but can be done through `awk`, `sed`, etc.

## Optimizing using G2O

This is done by using `g2o_viewer` which contains an optimizer as well

```bash
g2o_viewer ./given.g2o
```

### G2O Viewer

The following window opens, showing the vertices and loop closure constraints

![g2o_viewer vertices and loop closure](./results/q3/G2O/viewer_1.png)

Running 5 iterations of `gn_var_cholmod` optimizer gives the following resulting trajectory

![g2o_viewer after optimization](./results/q3/G2O/viewer_2.png)

This is saved using `File` > `Save` as `out.g2o`.

This is a much better trajectory by the looks of it.


## Intel and Sphere

The files `intel.g2o` (Intel research lab) and `sphere.g2o` were also optimized using `g2o_viewer` (which is more interactive than `g2o` CLI command). The results are shown below.

They were first copied to the folder `g2o_exp` using the following commands

```bash
cp ../../data/intel.g2o ./
cp ../../data/sphere.g2o ./
```


### Intel

The original file contains the following vertices and loop closures

![Intel initial vertices](./results/q3/G2O/intel_1.png)



After running `5` iterations of the same `gn_var_cholmod` method, the following was achieved

![Intel after optimization](./results/q3/G2O/intel_2.png)

This was saved as `intel_out.g2o`

### Sphere

The sphere file gave the following output

![Sphere un-optimized vertices](./results/q3/G2O/sphere_1.png)



This doesn't look like a sphere at all. Several experiments were run (between different optimizers and their parameters) to get the following optimized output

![Sphere optimized vertices](./results/q3/G2O/sphere_2.png)

This was obtained using the `Robust Kernel` setting (`Cauchy` with kernel width 3). The optimizer was `lm_var_cholmod` with `initialLambda=0.1`